In [21]:
## 对文件summerOly_athletes.csv清洗后的文件athletes_data.csv中的数据进行初步处理
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("athletes_data.csv")
data.head()

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
0,A Dijiang,M,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
2,Gunnar Aaby,M,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,No medal
3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Cornelia (-strannood),F,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,No medal


In [3]:
names = set(data["Name"])
nocs = set(data["NOC"])
events = set(data["Event"])

In [4]:
names_g = data.groupby("Name")
personal_data = {
    "Name": [],
    "NOC": [],
    "Gold": [],
    "Silver": [],
    "Bronze": [],
    "Medal": [],
}
for n in names:
    # print(n)
    tem = names_g.get_group(n)
    # print(tem)
    Gold_num = 0  # 该队员的金牌数
    Silver_num = 0  # 该队员的银牌数
    Bronze_num = 0  # 该队员的铜牌数
    for j in tem["Medal"]:
        if j == "Gold":
            Gold_num += 1
        elif j == "Silver":
            Silver_num += 1
        elif j == "Bronze":
            Bronze_num += 1
    personal_data["Name"].append(n)
    personal_data["NOC"].append(tem["NOC"].iloc[0])
    personal_data["Gold"].append(Gold_num)
    personal_data["Silver"].append(Silver_num)
    personal_data["Bronze"].append(Bronze_num)
    personal_data["Medal"].append(Gold_num + Silver_num + Bronze_num)

personal_data = pd.DataFrame(personal_data)
personal_data.head()

,Name,NOC,Gold,Silver,Bronze,Medal
0,orsteinn Hjlmarsson,ISL,0,0,0,0
1,Wolrad Eberle,GER,0,0,1,1
2,Bengt Frbom,SWE,0,0,0,0
3,Li Jingjing,CHN,0,0,0,0
4,Antonio Vargas,USA,0,0,0,0


In [5]:
personal_data.to_csv("personal_data.csv")

In [6]:
## 对国家进行分类
## （1） 参加比赛次数超过10次的国家
## （2） 参加比赛次数小于10次的国家
## （3） 参加比赛次数仅一两次的国家
## 其它因素对国家奖牌数量的影响：
## （1） 东道主效应
## （2） 伟大教练

In [ ]:
# 现进行国家分类

# 1 从运动员数据中获得国家列表
nocs
# 2 从运动员数据中获得每一届的参赛国家
# 先按照年份进行分类
years_g = data.groupby("Year")
years = set(data["Year"])
join_countrys = {i: list() for i in years}
for year in years:
    this_year = years_g.get_group(year)
    for j in this_year["NOC"]:
        # print(type(j))
        # print(type(join_countrys[year]))
        # print(type(join_countrys))
        # print(type(year))
        if j not in join_countrys[year]:
            join_countrys[year].append(j)
print(join_countrys)

{1920: ['DEN', 'FIN', 'NOR', 'EGY', 'GBR', 'SWE', 'ESP', 'USA', 'SUI', 'BEL', 'FRA', 'ITA', 'LUX', 'BRA', 'GRE', 'TCH', 'IND', 'MON', 'CHI', 'CAN', 'AUS', 'NED', 'RSA', 'POR', 'YUG', 'NZL', 'JPN', 'EST', 'ARG'], 1924: ['FIN', 'NOR', 'BUL', 'CHI', 'SUI', 'MON', 'GBR', 'FRA', 'BEL', 'HUN', 'POL', 'AUT', 'SWE', 'NED', 'EGY', 'MEX', 'ARG', 'TUR', 'ITA', 'USA', 'CUB', 'ESP', 'LUX', 'GRE', 'DEN', 'URU', 'AUS', 'TCH', 'LTU', 'EST', 'LAT', 'YUG', 'HAI', 'ROU', 'CAN', 'RSA', 'IRL', 'POR', 'BRA', 'JPN', 'IND', 'NZL', 'ECU', 'PHI', 'RUS'], 1928: ['NOR', 'EGY', 'USA', 'IND', 'SUI', 'GER', 'GBR', 'CAN', 'SWE', 'BEL', 'FRA', 'NED', 'ITA', 'MEX', 'FIN', 'DEN', 'JPN', 'LTU', 'ARG', 'ROU', 'ESP', 'CHI', 'LUX', 'POR', 'GRE', 'URU', 'AUT', 'IRL', 'POL', 'YUG', 'TCH', 'TUR', 'RSA', 'HUN', 'MON', 'CUB', 'LAT', 'ZIM', 'AUS', 'MLT', 'HAI', 'NZL', 'EST', 'PHI', 'PAN', 'BUL'], 1932: ['NED', 'USA', 'BEL', 'FRA', 'PHI', 'ITA', 'FIN', 'SWE', 'GBR', 'MEX', 'GER', 'IND', 'ESP', 'BRA', 'DEN', 'JPN', 'CAN', 'ARG', 'H

In [15]:
# 得到2028年可能参赛的国家有
join_country = join_countrys[2024]
# 3 利用data统计参赛国的的参赛次数
num_of_participate = {i: 0 for i in join_country}
# 按国家分类后依据是否有该国的成员来判断是否参了加本届奥运会并统计有多少次参加了奥运会
country_g = data.groupby("NOC")
for noc in join_country:
    country_participants = country_g.get_group(noc)
    num = len(set(country_participants["Year"]))
    num_of_participate[noc] = num

In [37]:
num_of_participate = pd.DataFrame(num_of_participate, index=[0])
num_of_participate

,ARM,COL,JAM,GAM,MEX,COM,IRQ,DJI,UAE,IRI,...,SAM,COD,SYR,PAN,SSD,NRU,PHI,SUI,PRK,SLO
0,8,22,19,11,25,8,16,10,11,19,...,11,12,15,19,3,8,23,31,11,9


In [42]:
# 进行分类
# 1 参加比赛次数超过10次的国家
country_10 = []
# 2 参加比赛次数小于10次大于5次的国家
country_5_10 = []
# 3 参加比赛次数小于5次的国家
country_1_5 = []

# 设立分类机
def classify_country(noc):
    if num_of_participate[noc].iloc[0] > 10:
        country_10.append(noc)
    elif num_of_participate[noc].iloc[0] > 5:
        country_5_10.append(noc)
    else:
        country_1_5.append(noc)
# 分类
for noc in join_country:
    classify_country(noc)
print(country_10)
print(country_5_10)
print(country_1_5)

['COL', 'JAM', 'GAM', 'MEX', 'IRQ', 'UAE', 'IRI', 'MAW', 'ARG', 'SGP', 'MAS', 'MTN', 'GAB', 'IND', 'ETH', 'CGO', 'PUR', 'ISV', 'JOR', 'OMA', 'GHA', 'LBA', 'ALG', 'CAY', 'GUY', 'ROU', 'BRN', 'CIV', 'BER', 'LUX', 'MAR', 'IRL', 'SUR', 'PAR', 'FRA', 'CUB', 'KUW', 'JPN', 'GER', 'BHU', 'BAN', 'CAM', 'AFG', 'LES', 'ISR', 'SWZ', 'GBR', 'SWE', 'VIE', 'MOZ', 'TPE', 'ITA', 'MLI', 'ESP', 'FIJ', 'CRC', 'UGA', 'BUL', 'MRI', 'BUR', 'GRN', 'BAR', 'SEY', 'MON', 'CYP', 'ANT', 'CHI', 'NIG', 'LIE', 'CHA', 'AUT', 'PER', 'SEN', 'QAT', 'BRA', 'NGR', 'BOL', 'BIZ', 'EGY', 'PAK', 'TUR', 'POL', 'MGL', 'BEN', 'ZIM', 'IVB', 'KOR', 'HUN', 'AND', 'GUI', 'ESA', 'SOL', 'THA', 'SUD', 'RSA', 'NOR', 'INA', 'KEN', 'DEN', 'RWA', 'ANG', 'TAN', 'EST', 'LTU', 'NZL', 'CHN', 'SMR', 'VEN', 'MLT', 'BEL', 'SOM', 'LBR', 'ZAM', 'AUS', 'KSA', 'TOG', 'DOM', 'SLE', 'HAI', 'FIN', 'BAH', 'TTO', 'USA', 'TUN', 'URU', 'MAD', 'CAF', 'POR', 'NEP', 'HKG', 'HON', 'PNG', 'ISL', 'ECU', 'SRI', 'NED', 'CAN', 'MYA', 'BOT', 'LAT', 'TGA', 'NCA', 'CMR'